In [51]:
import pandas as pd
import sqlite3
import copy

In [2]:
conn = sqlite3.connect("./Resources/gdp_olympic.sqlite")
cursor = conn.cursor()
# cursor.execute("SELECT year FROM winter")
# print(cursor.fetchall())

In [63]:
df = pd.read_sql_query("select * from summer_clean;", conn)
df

grouped = df.groupby(['year','country_code','medal']).count().reset_index()
grouped = grouped[['year','country_code','medal', 'id']]
grouped
pivot = pd.pivot_table(grouped, index=['year','country_code'], columns='medal')
pivot

pivot.columns = pivot.columns.get_level_values(1)
pivot

pivot = pivot.reset_index()
del pivot.columns.name
pivot.fillna(0, inplace=True)
pivot[pivot['country_code'] == GBR] # filter only to 2012 year in order to get graph done

year country_code  Bronze  Gold  Silver
0   1896          AUS     0.0   2.0     0.0
1   1896          AUT     2.0   2.0     1.0
2   1896          DEN     3.0   1.0     2.0
3   1896          FRA     2.0   5.0     4.0
4   1896          GBR     2.0   2.0     3.0
5   1896          GER     2.0  26.0     5.0
6   1896          GRE    22.0  10.0    20.0
7   1896          HUN     3.0   2.0     1.0
8   1896          SUI     0.0   1.0     2.0
9   1896          USA     2.0  11.0     7.0
10  1896          ZZX     2.0   2.0     2.0

In [96]:
# Testing
pivot[pivot['country_code'] == 'GBR'][:10]

year country_code  Bronze   Gold  Silver
4    1896          GBR     2.0    2.0     3.0
20   1900          GBR    10.0   48.0    20.0
34   1904          GBR     0.0    1.0     1.0
49   1908          GBR    68.0  144.0   135.0
67   1912          GBR    58.0   44.0    58.0
87   1920          GBR    17.0   55.0    35.0
109  1924          GBR    20.0   17.0    29.0
139  1928          GBR    24.0    6.0    25.0
167  1932          GBR    14.0    8.0    12.0
196  1936          GBR     8.0   12.0    16.0

In [64]:
pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1158 entries, 0 to 1157
Data columns (total 5 columns):
year            1158 non-null int64
country_code    1158 non-null object
Bronze          1158 non-null float64
Gold            1158 non-null float64
Silver          1158 non-null float64
dtypes: float64(3), int64(1), object(1)
memory usage: 45.3+ KB


In [57]:
import itertools
years = list(pivot['year'].unique())
countries = list(pivot['country_code'].unique())
countries = sorted(countries)
countries = countries[1:]

combinations = list(itertools.product(years,countries))
combinations

# Note, may need to get list of countries in the geoJSON in order to make sure that
# all countries are on the map
# Then check if there are any medals and if there aren't then maybe color different?

[(1896, 'AFG'),
 (1896, 'AHO'),
 (1896, 'ALG'),
 (1896, 'ANZ'),
 (1896, 'ARG'),
 (1896, 'ARM'),
 (1896, 'AUS'),
 (1896, 'AUT'),
 (1896, 'AZE'),
 (1896, 'BAH'),
 (1896, 'BAR'),
 (1896, 'BDI'),
 (1896, 'BEL'),
 (1896, 'BER'),
 (1896, 'BLR'),
 (1896, 'BOH'),
 (1896, 'BOT'),
 (1896, 'BRA'),
 (1896, 'BRN'),
 (1896, 'BUL'),
 (1896, 'BWI'),
 (1896, 'CAN'),
 (1896, 'CHI'),
 (1896, 'CHN'),
 (1896, 'CIV'),
 (1896, 'CMR'),
 (1896, 'COL'),
 (1896, 'CRC'),
 (1896, 'CRO'),
 (1896, 'CUB'),
 (1896, 'CYP'),
 (1896, 'CZE'),
 (1896, 'DEN'),
 (1896, 'DJI'),
 (1896, 'DOM'),
 (1896, 'ECU'),
 (1896, 'EGY'),
 (1896, 'ERI'),
 (1896, 'ESP'),
 (1896, 'EST'),
 (1896, 'ETH'),
 (1896, 'EUA'),
 (1896, 'EUN'),
 (1896, 'FIN'),
 (1896, 'FRA'),
 (1896, 'FRG'),
 (1896, 'GAB'),
 (1896, 'GBR'),
 (1896, 'GDR'),
 (1896, 'GEO'),
 (1896, 'GER'),
 (1896, 'GHA'),
 (1896, 'GRE'),
 (1896, 'GRN'),
 (1896, 'GUA'),
 (1896, 'GUY'),
 (1896, 'HAI'),
 (1896, 'HKG'),
 (1896, 'HUN'),
 (1896, 'INA'),
 (1896, 'IND'),
 (1896, 'IOP'),
 (1896, 

In [17]:
# Append gold medal count to geoJSON and then export
import json
import requests

response = requests.get("https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json")

geo = json.loads(response.text)

type(geo)

dict

In [12]:
data = [{"year":2010,"value":1},{"year":2011,"value":2}]
creteria = [{"year":2010,"insert":1},{"year":2011,"insert":2}]
for comb in creteria:
    creteria.filter(lambda x: x["year"] == )

[{'year': 2010, 'value': 1}, {'year': 2011, 'value': 2}]

In [83]:
def create_geo(i):
    try:
        year = pivot['year']
        country_code = pivot['country_code']

        num_gold = pivot[(year == i[0]) & (country_code == i[1])]['Gold'].iloc[0]
        num_silver = pivot[(year == i[0]) & (country_code == i[1])]['Silver'].iloc[0]
        num_bronze = pivot[(year == i[0]) & (country_code == i[1])]['Bronze'].iloc[0]
    #         print(num_gold, num_silver, num_bronze)
        
        country_info = list(filter(lambda x: x['id'] == i[1] , geo['features']))[0]
    #         country_info = next(country for country in geo['features'] if country["id"] == i[1])
    #         print(country_info)
        info1 = copy.deepcopy(country_info)

    #         medals_dict = {'gold':{str(year.iloc[0]):}

        info1['properties']['year'] = int(i[0])#pivot['year'].iloc[0]
        info1['properties']['gold'] = int(num_gold)
        info1['properties']['silver'] = int(num_silver)
        info1['properties']['bronze'] = int(num_bronze)
    #         features = features + [country_info]
    #     print(country_info)
#         if (info1.isna()):
#             return()
#         else:
        return(info1)
    except:
        pass
#     print(features)

In [53]:
# combinations1 = combinations[0]
combinations1 = [(1896, 'AUS'), (1900, 'AUS')]

In [81]:
combinations[:10]

[(1896, 'AFG'),
 (1896, 'AHO'),
 (1896, 'ALG'),
 (1896, 'ANZ'),
 (1896, 'ARG'),
 (1896, 'ARM'),
 (1896, 'AUS'),
 (1896, 'AUT'),
 (1896, 'AZE'),
 (1896, 'BAH')]

In [90]:
features = list(map(lambda x: create_geo(x),combinations))

features_final = []

for item in features:
    if item:
        features_final.append(item)
    else:
        pass


In [91]:
features[:10]

[None,
 None,
 None,
 None,
 None,
 None,
 {'type': 'Feature',
  'id': 'AUS',
  'properties': {'name': 'Australia',
   'year': 1896,
   'gold': 2,
   'silver': 0,
   'bronze': 0},
  'geometry': {'type': 'MultiPolygon',
   'coordinates': [[[[145.397978, -40.792549],
      [146.364121, -41.137695],
      [146.908584, -41.000546],
      [147.689259, -40.808258],
      [148.289068, -40.875438],
      [148.359865, -42.062445],
      [148.017301, -42.407024],
      [147.914052, -43.211522],
      [147.564564, -42.937689],
      [146.870343, -43.634597],
      [146.663327, -43.580854],
      [146.048378, -43.549745],
      [145.43193, -42.693776],
      [145.29509, -42.03361],
      [144.718071, -41.162552],
      [144.743755, -40.703975],
      [145.397978, -40.792549]]],
    [[[143.561811, -13.763656],
      [143.922099, -14.548311],
      [144.563714, -14.171176],
      [144.894908, -14.594458],
      [145.374724, -14.984976],
      [145.271991, -15.428205],
      [145.48526, -16.285672],


In [92]:
features_final[:10]

[{'type': 'Feature',
  'id': 'AUS',
  'properties': {'name': 'Australia',
   'year': 1896,
   'gold': 2,
   'silver': 0,
   'bronze': 0},
  'geometry': {'type': 'MultiPolygon',
   'coordinates': [[[[145.397978, -40.792549],
      [146.364121, -41.137695],
      [146.908584, -41.000546],
      [147.689259, -40.808258],
      [148.289068, -40.875438],
      [148.359865, -42.062445],
      [148.017301, -42.407024],
      [147.914052, -43.211522],
      [147.564564, -42.937689],
      [146.870343, -43.634597],
      [146.663327, -43.580854],
      [146.048378, -43.549745],
      [145.43193, -42.693776],
      [145.29509, -42.03361],
      [144.718071, -41.162552],
      [144.743755, -40.703975],
      [145.397978, -40.792549]]],
    [[[143.561811, -13.763656],
      [143.922099, -14.548311],
      [144.563714, -14.171176],
      [144.894908, -14.594458],
      [145.374724, -14.984976],
      [145.271991, -15.428205],
      [145.48526, -16.285672],
      [145.637033, -16.784918],
      [145

In [93]:

new_geo = {"type":"FeatureCollection","features":features_final}

import json

with open('result.json', 'w') as fp:
    json.dump(new_geo, fp)

In [20]:
new_geo = {"type":"FeatureCollection","features":[]}
features = []
for i in combinations:
    try:
        year = pivot['year']
        country_code = pivot['country_code']
        
        num_gold = pivot[(year == i[0]) & (country_code == i[1])]['Gold'].iloc[0]
        num_silver = pivot[(year == i[0]) & (country_code == i[1])]['Silver'].iloc[0]
        num_bronze = pivot[(year == i[0]) & (country_code == i[1])]['Bronze'].iloc[0]
#         print(num_gold, num_silver, num_bronze)
        
        country_info = list(filter(lambda x: x['id'] == i[1] , geo['features']))[0]
#         country_info = next(country for country in geo['features'] if country["id"] == i[1])
#         print(country_info)
        
#         medals_dict = {'gold':{str(year.iloc[0]):}
        
        country_info['properties']['year'] = int(i[0])#pivot['year'].iloc[0]
        country_info['properties']['gold'] = int(num_gold)
        country_info['properties']['silver'] = int(num_silver)
        country_info['properties']['bronze'] = int(num_bronze)
#         features = features + [country_info]
        features.append(country_info)
        print(features)
#         country_info = {}
#         new_geo['features'].update(country_info)
        
#         for country in geo['features']:
# #             print(country['id'])
# #             print(country_code)
#             if country['id'] == str(country_code):
# #                 print(country_code)
#                 country['properties']['year'][str(year)]['gold'] = num_gold
#                 country['properties']['year'][str(year)]['silver'] = num_silver
#                 country['properties']['year'][str(year)]['bronze'] = num_bronze
#                 break
#             else:
#                 pass
    except:
        pass

# print(type(new_geo))
# print(new_geo)
# print(features)



[{'type': 'Feature', 'id': 'AFG', 'properties': {'name': 'Afghanistan', 'year': 2012, 'gold': 0, 'silver': 0, 'bronze': 1}, 'geometry': {'type': 'Polygon', 'coordinates': [[[61.210817, 35.650072], [62.230651, 35.270664], [62.984662, 35.404041], [63.193538, 35.857166], [63.982896, 36.007957], [64.546479, 36.312073], [64.746105, 37.111818], [65.588948, 37.305217], [65.745631, 37.661164], [66.217385, 37.39379], [66.518607, 37.362784], [67.075782, 37.356144], [67.83, 37.144994], [68.135562, 37.023115], [68.859446, 37.344336], [69.196273, 37.151144], [69.518785, 37.608997], [70.116578, 37.588223], [70.270574, 37.735165], [70.376304, 38.138396], [70.806821, 38.486282], [71.348131, 38.258905], [71.239404, 37.953265], [71.541918, 37.905774], [71.448693, 37.065645], [71.844638, 36.738171], [72.193041, 36.948288], [72.63689, 37.047558], [73.260056, 37.495257], [73.948696, 37.421566], [74.980002, 37.41999], [75.158028, 37.133031], [74.575893, 37.020841], [74.067552, 36.836176], [72.920025, 36.720

[{'type': 'Feature', 'id': 'AFG', 'properties': {'name': 'Afghanistan', 'year': 2012, 'gold': 0, 'silver': 0, 'bronze': 1}, 'geometry': {'type': 'Polygon', 'coordinates': [[[61.210817, 35.650072], [62.230651, 35.270664], [62.984662, 35.404041], [63.193538, 35.857166], [63.982896, 36.007957], [64.546479, 36.312073], [64.746105, 37.111818], [65.588948, 37.305217], [65.745631, 37.661164], [66.217385, 37.39379], [66.518607, 37.362784], [67.075782, 37.356144], [67.83, 37.144994], [68.135562, 37.023115], [68.859446, 37.344336], [69.196273, 37.151144], [69.518785, 37.608997], [70.116578, 37.588223], [70.270574, 37.735165], [70.376304, 38.138396], [70.806821, 38.486282], [71.348131, 38.258905], [71.239404, 37.953265], [71.541918, 37.905774], [71.448693, 37.065645], [71.844638, 36.738171], [72.193041, 36.948288], [72.63689, 37.047558], [73.260056, 37.495257], [73.948696, 37.421566], [74.980002, 37.41999], [75.158028, 37.133031], [74.575893, 37.020841], [74.067552, 36.836176], [72.920025, 36.720

[{'type': 'Feature', 'id': 'AFG', 'properties': {'name': 'Afghanistan', 'year': 2012, 'gold': 0, 'silver': 0, 'bronze': 1}, 'geometry': {'type': 'Polygon', 'coordinates': [[[61.210817, 35.650072], [62.230651, 35.270664], [62.984662, 35.404041], [63.193538, 35.857166], [63.982896, 36.007957], [64.546479, 36.312073], [64.746105, 37.111818], [65.588948, 37.305217], [65.745631, 37.661164], [66.217385, 37.39379], [66.518607, 37.362784], [67.075782, 37.356144], [67.83, 37.144994], [68.135562, 37.023115], [68.859446, 37.344336], [69.196273, 37.151144], [69.518785, 37.608997], [70.116578, 37.588223], [70.270574, 37.735165], [70.376304, 38.138396], [70.806821, 38.486282], [71.348131, 38.258905], [71.239404, 37.953265], [71.541918, 37.905774], [71.448693, 37.065645], [71.844638, 36.738171], [72.193041, 36.948288], [72.63689, 37.047558], [73.260056, 37.495257], [73.948696, 37.421566], [74.980002, 37.41999], [75.158028, 37.133031], [74.575893, 37.020841], [74.067552, 36.836176], [72.920025, 36.720

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [21]:
features

[{'type': 'Feature',
  'id': 'AFG',
  'properties': {'name': 'Afghanistan',
   'year': 2012,
   'gold': 0,
   'silver': 0,
   'bronze': 1},
  'geometry': {'type': 'Polygon',
   'coordinates': [[[61.210817, 35.650072],
     [62.230651, 35.270664],
     [62.984662, 35.404041],
     [63.193538, 35.857166],
     [63.982896, 36.007957],
     [64.546479, 36.312073],
     [64.746105, 37.111818],
     [65.588948, 37.305217],
     [65.745631, 37.661164],
     [66.217385, 37.39379],
     [66.518607, 37.362784],
     [67.075782, 37.356144],
     [67.83, 37.144994],
     [68.135562, 37.023115],
     [68.859446, 37.344336],
     [69.196273, 37.151144],
     [69.518785, 37.608997],
     [70.116578, 37.588223],
     [70.270574, 37.735165],
     [70.376304, 38.138396],
     [70.806821, 38.486282],
     [71.348131, 38.258905],
     [71.239404, 37.953265],
     [71.541918, 37.905774],
     [71.448693, 37.065645],
     [71.844638, 36.738171],
     [72.193041, 36.948288],
     [72.63689, 37.047558],
     

In [131]:
type(new_geo)

dict